In [6]:
import pymongo as pm
import os
import sys
import numpy
import pandas as pd
import json
from IPython.display import clear_output

## connect to db

#### (1) establish tunnel to lab server: 
- Run this in terminal: `ssh -fNL 27017:127.0.0.1:27017 USER@cogtoolslab.org`
OR

- Add this to your .bashrc or (.zshrc):
`alias tunnel_cogtoolslab="ssh -fNL 27017:127.0.0.1:27017 USER@cogtoolslab.org"`

#### (2) establish connection to target db and collection

In [75]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## experiment server ip address

conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db['block-construction-silhouette']

# raw stimulus data
path_to_interesting_structures = 'interesting_structures'


In [35]:
# print list of database names 
print('dbnames:')
print(conn.list_database_names())

# now print list of collection names within stimuli db
print('colnames:')
print(conn['stimuli'].list_collection_names())

dbnames:
['3dObjects', 'admin', 'block_construction', 'collabdraw', 'collabdraw_recog', 'config', 'drawbase', 'gallerize', 'human_compression', 'iteratedNumber', 'local', 'semantic_parts', 'stimuli']
colnames:
['svg_annotation_sketchpad_basic_allcats', 'collabdraw_collab8_recog', 'semantic_parts_graphical_conventions', 'photodraw2']


#### Setting up the stimulus dictionary
- In the design of silhouette-study-1, there are two conditions: ['external', 'internal'], which will be manipulated between participant. (Or within participant, even?)
- At the beginning of each experimental session, all of the trials to construct a single session are fetched from the database. 
- Each trial dictionary needs to contain the following attributes: 
    - `target`: List of blocks generated elsewhere in this `stimuli/` dir
    - `condition`: "external" can be a placeholder
    - `games`: Empty games list that will be populated by `getstims` inside `store.js` as games fetch this type of game

In [102]:
file_list = os.listdir(path_to_interesting_structures)
trial_list = []
for i,f in enumerate(file_list):
    x = pd.read_json(os.path.join(path_to_interesting_structures,f),orient='records')
    this_targetBlocks = list(x['blocks'].values)
    this_targetName = f.split('.')[0]
    this_trial = {'targetBlocks':this_targetBlocks,
                  'targetName': this_targetName,            
                  'condition':'external',
                  'games':[]}
    trial_list.append(this_trial) ## this is not particularly elegant -- I AM NOT PROUD, @jefan.
print('Our trial_list is {} entries long.'.format(len(trial_list)))

Our trial_list is 12 entries long.


In [105]:
## this is ALL not particularly elegant
trial_df = pd.DataFrame(trial_list)
## save out for redundancy
trial_df.to_json('stimDict.json',orient='records')
## ok, just make sure you can read it back in, and use this var to insert into mongo below
stimDict = pd.read_json('stimDict.json')
print('Our stimDict is {} entries long.'.format(stimDict.shape[0]))

Our stimDict is 12 entries long.


In [106]:
## now, iterate through each record and insert into mongo
## loop through list of records and insert each into collection
reallyRun = 1
if reallyRun:
    for ind,row in stimDict.iterrows():
        ## extract dictionary representation of each trial 
        this_record = row.to_dict()        
        coll.insert_one(this_record)
        print('Inserted {} records.'.format(ind+1))
        clear_output(wait=True)
else:
    print('Did not insert any new data.')

Inserted 12 records.


### inspect and validate collection

In [107]:
print('There are {} records in this collection.'.format(coll.estimated_document_count()))

There are 12 records in this collection.


In [109]:
## Now let's look at one of the records
coll.find_one()

{'_id': ObjectId('5daf4fc41faa5dacd8e53e11'),
 'condition': 'external',
 'games': [],
 'targetBlocks': [{'width': 4, 'height': 2, 'x': 0, 'y': 0},
  {'width': 2, 'height': 2, 'x': 5, 'y': 0},
  {'width': 2, 'height': 1, 'x': 0, 'y': 2},
  {'width': 1, 'height': 2, 'x': 2, 'y': 2},
  {'width': 4, 'height': 2, 'x': 4, 'y': 2},
  {'width': 1, 'height': 2, 'x': 0, 'y': 3},
  {'width': 1, 'height': 2, 'x': 1, 'y': 3},
  {'width': 1, 'height': 2, 'x': 2, 'y': 4},
  {'width': 1, 'height': 2, 'x': 5, 'y': 4},
  {'width': 2, 'height': 1, 'x': 6, 'y': 4},
  {'width': 1, 'height': 2, 'x': 1, 'y': 5},
  {'width': 1, 'height': 2, 'x': 6, 'y': 5},
  {'width': 1, 'height': 2, 'x': 7, 'y': 5},
  {'width': 1, 'height': 2, 'x': 2, 'y': 6},
  {'width': 1, 'height': 2, 'x': 5, 'y': 6},
  {'width': 2, 'height': 1, 'x': 6, 'y': 7}],
 'targetName': 'hand_selected_007'}

In [113]:
conn['stimuli'].collection_names()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['svg_annotation_sketchpad_basic_allcats',
 'collabdraw_collab8_recog',
 'semantic_parts_graphical_conventions',
 'block-construction-silhouette',
 'photodraw2']